# PRAIA MINING 

Hoje vamos aprender a minerar dados com a conversa do grupo!

O primeiro experimento vai ser a geração da base, a exploração da mesma e por fim a geração de uma wordcloud.

Se você ainda não exportou a conversa do grupo, faça isso seguindo indo nas opções da conversa e clicando em mais e depois em exportar conversa.

No nosso caso a conversa está salva na mesma pasta que o código e com o nome de 'conversa.txt'. Se você salvar em um lugar diferente, basta modificar abaixo.
O código a seguir será o responsável por abrir o arquivo e salvar seu conteúdo na variável wpp_test.

Mas antes de rodar qualquer uma das células, **ATENÇÃO!**

Pelo terminal, dentro do repositório, instale os requisitos usando o comando `pip install -r requirements.txt`. Na verade, talvez você não precise desse passo caso esteja rodando o notebook, porque o Jupyter vai rodar o código no próprio servidor. Mas caso precise, já sabe o que fazer.

E lembre-se de rodar as células em ordem, caso contrário, o código pode se comportar de maneira inesperada.

In [ ]:
# Abre o arquivo
wpp_text = open('conversa.txt').read()

# Exibe os primeiros 1000 caracteres para nos dar uma ideia do conteúdo da base. 
# Também vale a pena dar uma olhada rápida no arquivo para ter uma ideia do que estaremos lidando.
wpp_text[:1000]

Agora que já extraímos o texto do arquivo da conversa precisamos transformar esses dados em algo mais fácil de manipular do que uma sequência de caracteres!

De cara já podemos ver que toda linha possui uma estrutura: <data\> <horário> - <quem enviou a mensagem\>: <mensagem\>. Agora, o que queremos é transformar a conversa em uma tabela, onde cada uma dessas informações presente numa linha vai ficar em uma coluna. 

Nessa parte, como todas as outras, é importante explorar a base, usando por exemplo a função `split()` em diferentes linhas para encontrar uma forma de separar as informações de uma maneira que funcione para todas as linhas.

In [ ]:
# o primeiro split quebra o texto em uma lista de linhas, usando como divisor o carácter de quebra de linha ('\n') 
# o segundo split quebra a linha 250 em uma lista, usando o carácter espaço (' ')
wpp_text.split('\n')[250].split(' ')

Seguindo a ideia de explorar a base, aqui podemos ter uma ideia que quantas linhas temos

In [ ]:
len(wpp_text.split('\n'))

Depois de explorar a base, precisamos escolher uma forma de transforma-la em uma tabela, tabela essa que será um DataFrame, uma das estrutura de dados implementadas pelo pacote Pandas.

Outra estrutura implementada pelo Pandas é a Series, que também utilizaremos aqui, já que ela possue um método para gerar um DataFrame a partir de expressões regulares, o `Series.str.extract()`

In [ ]:
import pandas as pd

# wpp_text.split('\n') quebra o texto em uma lista de linhas, 
# e pd.Series() transforma essa lista em uma Series, que é uma estrutura mais parecida com um vetor
wpp_series = pd.Series(wpp_text.split('\n'))

# essa linha imprime o resultado
wpp_series

A parte da geração do dataframe a partir de uma expressão regular segue logo abaixo mas talvez você olhe a primeira linha e se pergunte: que p***a tá acontecendo aqui?

Relaxe, eu explico.

O método `Series.str.extract()` transforma os grupos de captura de uma expressão regular nas colunas do nosso DataFrame.

E os grupos de captura são as partes da expresão regular que estão envolvidas em parênteses! 

Se lique no que cada um deles vai capturar:

 - (\d{2}\/\d{2}\/\d{4})
 
 Essa é a captura da data. 
 
 **\d{2}** captura uma sequência de dois dígitos, se houver 4 no lugar do 2, 4 dígitos são capturados.
 
 **\/** captura o caráctere \ (barra).
 
 Combinando esses elementos, conseguimos capturar uma sequência de 2 dígitos, seguida de \, seguida de outra de 2 dígitos, seguida de barra, seguida de uma sequência de 4 **( ͡° ͜ʖ ͡°)** 
 
 Ou seja, uma data.

 - Depois desse primeiro grupo, há um espaço, indicando que ante as strings capturadas deve haver uma espaço

 - (\d{2}:\d{2})

 A diferença desse grupo para o primeiro é que aqui deve haver : entre duas sequências de 2 dígitos, ou seja, hora.

 - Novamente um espaço, seguido de - e de outro espaço

 - (.*?:)

 Captura de nome ou número de contato

 . (ponto) irá casar com qualquer caractere, 
 
 \+ (mais) determina que deve haver um ou mais caracteres, 
 
 ? (interrogação) diz que a captura não deve ser gulosa, ou seja, assim que a expressão for satisfeita, a busca deve parar, 
 
 : (dois pontos) deve ser o último da string capturada pelo grupo.

 - (.*)

 Captura da mensagem, que vai ser qualquer caractere que venha depois do : (dois pontos)




In [ ]:
df = wpp_series.str.extract(r'(\d{2}\/\d{2}\/\d{4}) (\d{2}:\d{2}) - (.+?:) (.*)')

print(df)

Às vezes nossa base pode vir com algumas inconscistências, como dados faltando. No caso da nossa, algums linhas vieram com NaN, valores que podemos descartar com o método `DataFrame.dropna()`

Ao final da saída que será imprimida, é possivel ver que a quantidade de linhas da tabela diminuiu

In [ ]:
df = df.dropna()
print(df)

Esse passo será útil quando formos manipular as colunas: dar nomes a elas.

In [ ]:
df.columns=['date', 'time', 'sender', 'message']
print(df)

Com as colunas nomeadas podemos facilmente, por exemplo, selecionar a coluna sender (que guarda o nome do contato que enviou a mensagem) e usar nela o método unique(), nos retorna uma lista de valores únicos.

In [ ]:
print(df.sender.unique())

Caso sua base venha com algum contato sem nome (somente com o número), ou deseje renomear o sender por algum outro motivo, faça como foi feito aqui.

In [ ]:
# Nome e númerosão são somente representativos, por motivos de privacidade
df.sender[df.sender == '+55 71 9999-9999:'] = 'Fulano da Silva Júnior'

E se quisermos saber quantos mensagens cada contato enviou, qual foi a mensagem mais enviada por cada um, etc etc?

Podemos usar o método `groupby()` passando nossa coluna 'sender' como parêmetro, o que vai agrupar as linhas com base nos valores dessa coluna, e em cima desse resultado usar o método `describe()`, que vai resumir os dados em números, como média, número de mensagens, mensagem mais enviada e algumas outras informações.

In [ ]:
df.groupby('sender').describe()

E agora que temos o número de mensagens enviadas podemos usar o método `sort_values()` para ordenar os contatos ('sender') em sentido decrescente (ascending=False) usando a coluna 'count' (by='count') como critério de ordenação.

In [ ]:
df.groupby('sender').describe().date.sort_values(by='count', ascending=False)

Mas ainda não acabamos e tratar a base! ~~E quando você chegar ao final desse notebook vai ver que ainda haverão coisas a serem exploradas~~

Nossa base tem mensagens que eram arquivos de mídia, mas esses arquvos não estão na nossa base e saber que eles foram enviados não vai nos servir pra nada, por enquanto. 

O que fazemos então? Geramos uma cópia do DataFrame e retiramos as linhas onde as mensagens são <Arquivo de mídia oculto> com o método drop, que vai receber como parêmetro uma lista com os índices que serão excluídos ~~tô com preguiça de explicar melhor agora~~


In [ ]:
df_no_media = df.copy()

df_no_media = df_no_media.drop(df_no_media[df_no_media.message == '<Arquivo de mídia oculto>'].index).reset_index()
df_no_media


Agora finalmente vamos gerar a wordcloud! 
Usamos a classe WordCloud do pacote wordcloud e passamos como parêmetro uma lista de stopwords (palavras muito comuns que nada agragarão à nossa visualização)

In [ ]:
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# Gera a wordcloud
wc = WordCloud(max_words=80, background_color="white").generate(' '.join(df_no_media.message))

# Plota a wordcloud como imagem
plt.figure(figsize=(15,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Aqui pegamos uma lista de stopwords presente no pacote nltk
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords[:10]

In [ ]:
# Repetimos o processo anterior, mas dessa vez com as stopwords do nltk
wc = WordCloud(stopwords=stopwords, max_words=80, background_color="white").generate(' '.join(df_no_media.message))

plt.figure(figsize=(15,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

A partir dessa imagem conseguimos ver que ainda estão presentes alguma palavram que não são interessantes. Não tem problema, podemos acrescentá-las à nossa lista de stopwords

In [ ]:

# Stopwords identificadas a olho e adicinadas na mão grande
new_stopwords = stopwords + ['aí', 'né', 'pra', 'pq', 'tá', 'faz', 'pq', 'se', 'vc', 'https', 'www', 'tbm', 'voce', 'você', 'ia', 'já', 'tava']

# Stopwords presentes em outra lista que encontrei na internet
other_list = open('stopwords-pt.txt').read().split('\n')

# Stopwords finais, juntando todas as outras
final_stopwords = new_stopwords + other_list

# Geramos e plotamos a WordCloud novamente
wc = WordCloud(stopwords=final_stopwords, max_words=100, background_color="white").generate(' '.join(df_no_media.message))

plt.figure(figsize=(15,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

Por último salvamos a imagem gerada num arquivo png.

In [ ]:
wc.to_file('praia_wordcloud.png')

Mas e agora, o que mais pode ser feito?

Ao olhar melhor o arquivo das conversas, vi que mensagens com quebra de linha têm seu conteúdo desprezado pelo método `Series.str.extract()`, por causa do jeito que a expressão regular foi escrita. Como peegar essas mensagens quebradas?

Outras informações que podem ser interessantes são as mudanças de nome do grupo, mudanças de número, mudança de descrição e menções, que também não são extraídas.

Além disso, ficam aqui mais umas sugestões
- Gerar uma nuvem de palavras para cada membro do grupo
- Unificar algumas palavras, como kkkk com diferentes quantidades de k, ou considerar as palavras como case insensitive

E é isso, tá aí a zorra do código :)
